In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('db_basic_queries(1).sqlite3')

Cantiddad min max y avg 

¿Cuáles son las ciudades únicas de los clientes (no nulas), ordenadas alfabéticamente? Muestre los resultados(filas) 2,3 y 4, es decir, muestre tres resultados a partir del segundo.

In [ ]:
query = """
SELECT estado, 
SUM(cantidad), 
MIN(cantidad), 
 MAX(cantidad), 
AVG(cantidad) 
From pedidos GROUP BY estado

"""
pd.read_sql_query(query, conn)

,estado,SUM(cantidad),MIN(cantidad),MAX(cantidad),AVG(cantidad)
0,CANCELADO,2,2,2,2.0
1,ENTREGADO,6,1,3,2.0
2,ENVIADO,2,1,1,1.0
3,PENDIENTE,2,1,1,1.0


In [ ]:
query = """
SELECT Ciudad FROM clientes 
WHERE Ciudad IS NOT NULL 
ORDER BY Ciudad ASC 

"""
pd.read_sql_query(query, conn)

,ciudad
0,Bogotá
1,Bogotá
2,Cali
3,Medellín


In [5]:
query = """
SELECT * FROM clientes

"""
pd.read_sql_query(query, conn)

,cliente_id,nombre,ciudad,edad,fecha_registro
0,1,Ana Gómez,Bogotá,34,2025-01-15
1,2,Luis Pérez,Medellín,28,2025-03-22
2,3,María Ruiz,Cali,41,2024-11-07
3,4,Juan Torres,None,22,2025-07-01
4,5,Carla León,Bogotá,29,2025-06-18


In [9]:
query = """
SELECT DISTINCT producto FROM pedidos
"""

pd.read_sql_query(query, conn)

,producto
0,Laptop
1,Mouse
2,Teclado
3,Monitor
4,Smartphone
5,Tablet
6,Auriculares
7,Webcam


In [ ]:
# Deme la lista de los ultimos 3 clientes registrados
query = """
SELECT
  nombre, fecha_registro
FROM
  clientes
ORDER BY fecha_registro DESC
LIMIT 3
"""
pd.read_sql_query(query, conn)

In [ ]:
# Cuales son los clientes que ordenes de precio unitario mayor a 1000
query = """
SELECT DISTINCT
  A.nombre
FROM
  clientes A
  INNER JOIN pedidos B
    ON A.cliente_id = B.cliente_id
WHERE B.precio_unit > 1000
"""
pd.read_sql_query(query, conn)

In [2]:
# Muestre la tabla de pedidos pero creando la columna categoria_valor, donde tiene que
# devolver ALTO en el caso en el que el precio unitario supere 1000, para todo caso contrario
# colocar BAJO.
query = """
SELECT
  precio_unit,
  CASE
    WHEN precio_unit > 1000
      THEN 'ALTO'
    WHEN precio_unit < 1000 AND precio_unit >= 300
      THEN 'NORMAL'
  ELSE 'BAJO'
  END AS categoria_valor
FROM
  pedidos
"""
pd.read_sql_query(query, conn)

,precio_unit,categoria_valor
0,3500.0,ALTO
1,45.5,BAJO
2,80.0,BAJO
3,220.0,BAJO
4,1200.0,ALTO
5,300.0,NORMAL
6,65.0,BAJO
7,70.0,BAJO


In [20]:
# quiero una tabla que me agrupe id cuantos items hay con CTE
query = """
WITH  CantidadPorCliente AS 
(
 SELECT 
  cliente_id, 
  sum(cantidad) as itemsTotales 
 FROM pedidos
 WHERE cliente_id IS NOT NULL 
 Group by cliente_id
)

SELECT 
b.cliente_id,
b.nombre,
a.itemsTotales
FROM CantidadPorCliente A
Inner Join clientes B
on A.cliente_id = B.cliente_id

"""
# Define una sub queri y se guarda tipo variable 

pd.read_sql_query(query, conn)

,cliente_id,nombre,itemsTotales
0,1,Ana Gómez,3
1,2,Luis Pérez,2
2,3,María Ruiz,2
3,4,Juan Torres,1
4,5,Carla León,3


In [24]:
# PARTITION BY es una de las cláusulas utilizadas en las funciones ventana.
# PARITION BY ME DIFERENCIA POR BARIABLES LAS CONDICIONES DE VENTANA 


query = """
WITH  CantidadPorCliente AS 
(
 SELECT 
  cliente_id, 
  sum(cantidad) as itemsTotales 
 FROM pedidos
 WHERE cliente_id IS NOT NULL 
 Group by cliente_id
)

SELECT 
 b.cliente_id,
 b.nombre,
 a.itemsTotales
RANK() OVER (ORDER BY a.itemsTotales)  rank
FROM CantidadPorCliente A
Inner Join clientes B
on A.cliente_id = B.cliente_id
"""

pd.read_sql_query(query, conn)
  

DatabaseError: Execution failed on sql '
WITH  CantidadPorCliente AS 
(
 SELECT 
  cliente_id, 
  sum(cantidad) as itemsTotales 
 FROM pedidos
 WHERE cliente_id IS NOT NULL 
 Group by cliente_id
)

SELECT 
 b.cliente_id,
 b.nombre,
 a.itemsTotales
RANK() OVER (ORDER BY a.itemsTotales)  rank
FROM CantidadPorCliente A
Inner Join clientes B
on A.cliente_id = B.cliente_id
': near "(": syntax error